In [1]:
!pip install pycaret[full]
!pip install jinja2
!pip install tqdm
!pip install lightgbm
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255.9 MB 19 kB/s 
     |████████████████████████████████| 177 kB 36.0 MB/s 
     |████████████████████████████████| 132 kB 53.9 MB/s 
     |████████████████████████████████| 377 kB 45.5 MB/s 
     |████████████████████████████████| 59.4 MB 1.3 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 348 kB 52.3 MB/s 
     |████████████████████████████████| 312 kB 48.4 MB/s 
     |████████████████████████████████| 569 kB 56.4 MB/s 
     |████████████████████████████████| 5.3 MB 40.0 MB/s 
     |████████████████████████████████| 40 kB 5.1 MB/s 
     |████████████████████████████████| 100 kB 7.8 MB/s 
     |████████████████████████████████| 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import

In [2]:
from pycaret import regression

from xgboost import XGBRegressor

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

### Prepare the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe


In [4]:
train_data = pd.read_csv('../../00_original_data/FIFA_train.csv')

#### Preprocess

In [5]:
#contract_until 변수 int 형으로 변환
# 계약 연도만 추출
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:]
    return int(string)


train_data['contract_until'] = train_data['contract_until'].apply(func)

In [ ]:
# 문자형 변수 label 인코딩 진행
train_data['continent'] = train_data['continent'].astype('category').cat.codes
train_data['position'] = train_data['position'].astype('category').cat.codes
train_data['prefer_foot'] = train_data['prefer_foot'].astype('category').cat.codes

In [7]:
minmax_features = ['reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']
minmax_transformer = MinMaxScaler() # StandardScaler() # RobustScaler()

train_data[minmax_features] = minmax_transformer.fit_transform(train_data[minmax_features])

### Set up PyCaret Regression Model

In [8]:
model = regression.setup(data=train_data[train_data.keys().drop(['id', 'name'])], 
              target='value', 
              session_id=42) # Random seed 

,Description,Value
0,session_id,42
1,Target,value
2,Original Data,"(8932, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6252, 16)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='value',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
              

In [9]:
xgb = regression.create_model('xgboost')
xgb_tuned = regression.tune_model(xgb, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,307797.4688,5.029617e+11,7.091979e+05,0.9839,0.3669,0.2148
1,288208.3750,7.058097e+11,8.401248e+05,0.9756,0.4307,0.2669
2,286800.3438,4.956908e+11,7.040531e+05,0.9782,0.4020,0.2954
3,241701.7656,5.304565e+11,7.283244e+05,0.9800,0.3746,0.2244
4,285859.9062,7.786807e+11,8.824288e+05,0.9801,0.4753,0.2586
5,266931.1562,3.331395e+11,5.771824e+05,0.9878,0.5119,0.3203
6,339482.4688,1.254896e+12,1.120221e+06,0.9729,0.4105,0.2268
7,344304.8438,1.471125e+12,1.212899e+06,0.9700,0.4160,0.3064
8,289584.6875,5.383821e+11,7.337452e+05,0.9779,0.4639,0.3188


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.4, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=30, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=42, reg_alpha=1e-06, ...)
INFO:logs:tune_model() succesfully completed......................................


In [10]:
top_12_models = regression.compare_models(sort='RMSE', n_select=12)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.462430e+05,3.687808e+11,5.579822e+05,0.9900,0.1338,0.0725,2.174
gbr,Gradient Boosting Regressor,2.304130e+05,5.667691e+11,7.235177e+05,0.9829,0.3390,0.2052,0.358
lightgbm,Light Gradient Boosting Machine,1.755717e+05,6.667764e+11,7.615755e+05,0.9813,0.1263,0.0791,0.152
rf,Random Forest Regressor,1.817555e+05,6.953976e+11,7.941151e+05,0.9801,0.0999,0.0576,1.451
et,Extra Trees Regressor,1.867513e+05,7.733633e+11,8.390996e+05,0.9773,0.1093,0.0589,1.436
xgboost,Extreme Gradient Boosting,1.778179e+05,9.039069e+11,8.452764e+05,0.9746,0.1200,0.0653,0.755
dt,Decision Tree Regressor,2.373176e+05,1.382579e+12,1.140669e+06,0.9570,0.1329,0.0651,0.033
ada,AdaBoost Regressor,1.787657e+06,4.432732e+12,2.094241e+06,0.8538,1.5853,5.1668,0.281
llar,Lasso Least Angle Regression,1.811173e+06,1.067001e+13,3.234205e+06,0.6658,1.3209,4.6890,0.019
lr,Linear Regression,1.811267e+06,1.067002e+13,3.234209e+06,0.6658,1.3210,4.6894,0.032


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:[<catboost.core.CatBoostRegressor object at 0x7f47af146150>, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False), LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
          

In [11]:
top_12_models

 GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False),
 LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               rand

In [12]:
blended12 = regression.blend_models(top_12_models, 10, optimize='RMSE')
regression.predict_model(blended12)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,679436.0264,1.877861e+12,1.370351e+06,0.9397,0.7353,0.9956
1,636446.3381,1.163160e+12,1.078499e+06,0.9598,0.8559,1.3367
2,591308.8423,9.249261e+11,9.617308e+05,0.9593,0.8154,1.3442
3,627684.7573,1.157704e+12,1.075967e+06,0.9563,0.8449,1.2257
4,724192.5965,2.949423e+12,1.717388e+06,0.9247,0.8221,1.2393
5,653495.6313,1.816142e+12,1.347643e+06,0.9334,0.9706,1.8169
6,726140.5196,2.871320e+12,1.694497e+06,0.9380,0.8738,1.5441
7,731942.3471,3.227542e+12,1.796536e+06,0.9341,0.8289,1.2216
8,616011.4797,1.132096e+12,1.064000e+06,0.9535,0.7324,1.2697


INFO:logs:create_model_container: 23
INFO:logs:master_model_container: 23
INFO:logs:display_container: 5
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7f47ae513450>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,715002.0243,2.969464e+12,1.723213e+06,0.9231,0.877,1.662


,age,continent,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,value,Label
0,0.541667,4,2,1,0.00,0.425532,0.413043,0.25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,600000.0,7.714620e+05
1,0.333333,4,2,1,0.00,0.425532,0.478261,0.50,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,975000.0,1.973438e+06
2,0.875000,2,1,1,0.00,0.531915,0.521739,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,210000.0,6.793189e+05
3,0.375000,2,0,1,0.00,0.425532,0.478261,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,1.169653e+06
4,0.250000,0,2,1,0.00,0.531915,0.673913,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4400000.0,4.574366e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,0.458333,2,2,1,0.00,0.468085,0.456522,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.451871e+06
2676,0.541667,2,3,1,0.25,0.531915,0.521739,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,5.006582e+06
2677,0.333333,4,3,0,0.00,0.382979,0.434783,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,675000.0,1.067117e+06
2678,0.166667,2,3,1,0.00,0.276596,0.478261,0.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,6.825654e+05


In [13]:
blended5 = regression.blend_models(top_12_models[:5], 10, optimize='RMSE')
regression.predict_model(blended5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,165883.5213,2.627549e+11,5.125962e+05,0.9916,0.0972,0.0617
1,161308.5491,5.655424e+11,7.520255e+05,0.9805,0.1515,0.0698
2,138126.0852,2.316929e+11,4.813449e+05,0.9898,0.1244,0.0721
3,135074.9662,2.413721e+11,4.912963e+05,0.9909,0.1034,0.0667
4,159823.1084,5.100625e+11,7.141866e+05,0.9870,0.1119,0.0652
5,146122.5218,3.009286e+11,5.485696e+05,0.9890,0.3068,0.0957
6,222023.2390,1.072324e+12,1.035531e+06,0.9768,0.1571,0.0712
7,223599.0459,1.493255e+12,1.221988e+06,0.9695,0.1316,0.0745
8,153430.8934,3.742291e+11,6.117427e+05,0.9846,0.1239,0.0732


INFO:logs:create_model_container: 24
INFO:logs:master_model_container: 24
INFO:logs:display_container: 7
INFO:logs:VotingRegressor(estimators=[('catboost',
                             <catboost.core.CatBoostRegressor object at 0x7f47aa4c9e90>),
                            ('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                         

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,167367.2787,1.051183e+12,1.025272e+06,0.9728,0.1635,0.0812


,age,continent,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,contract_until_2018,contract_until_2019,contract_until_2020,contract_until_2021,contract_until_2022,contract_until_2023,contract_until_2024,contract_until_2025,value,Label
0,0.541667,4,2,1,0.00,0.425532,0.413043,0.25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,600000.0,6.269435e+05
1,0.333333,4,2,1,0.00,0.425532,0.478261,0.50,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,975000.0,9.446668e+05
2,0.875000,2,1,1,0.00,0.531915,0.521739,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,210000.0,4.540570e+05
3,0.375000,2,0,1,0.00,0.425532,0.478261,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,800000.0,7.864660e+05
4,0.250000,0,2,1,0.00,0.531915,0.673913,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4400000.0,4.167186e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,0.458333,2,2,1,0.00,0.468085,0.456522,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1100000.0,1.065533e+06
2676,0.541667,2,3,1,0.25,0.531915,0.521739,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3100000.0,2.963015e+06
2677,0.333333,4,3,0,0.00,0.382979,0.434783,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,675000.0,6.600438e+05
2678,0.166667,2,3,1,0.00,0.276596,0.478261,0.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325000.0,3.560804e+05
